<a href="https://colab.research.google.com/github/Janina712/MLTSA_FinalProject/blob/main/tryouts/Feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import numpy as np 
import pandas as pd
import os
import pylab as pl
from pathlib import Path
from collections import Counter
from scipy.stats import entropy
import matplotlib.mlab
import statistics
import scipy.stats as stats

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
cd /content/drive/MyDrive/MLTSA22/Final_Project

/content/drive/MyDrive/MLTSA22/Final_Project


In [5]:
!ls /content/drive/MyDrive/MLTSA22/Final_Project

00000258_s002_t000.edf
00010563_s002_t004.pkl
all_tuh_eeg_seizure_links.txt
dev__01_tcp_ar__002__00000258__s002_2003_07_21__00000258_s002_t000.edf
dev__01_tcp_ar__002__00000258__s003_2003_07_22__00000258_s003_t001.edf
files_raw_channels_small_df.pkl
isip_tuh_eeg
seizures_types_v01.xlsx
temp


In [10]:
working_folder = Path('/content/drive/MyDrive/MLTSA22/Final_Project/')

In [13]:
small = pd.read_pickle(
    working_folder.joinpath('00010563_s002_t004.pkl')
)

ValueError: ignored

In [14]:
columns = ['EEG FP1-REF', 'EEG FP2-REF', 'EEG F3-REF', 'EEG F4-REF',
       'EEG C3-REF', 'EEG C4-REF', 'EEG P3-REF', 'EEG P4-REF', 'EEG O1-REF',
       'EEG O2-REF', 'EEG F7-REF', 'EEG F8-REF', 'EEG T3-REF', 'EEG T4-REF',
       'EEG T5-REF', 'EEG T6-REF', 'EEG FZ-REF', 'EEG CZ-REF', 'EEG PZ-REF',
       'EEG A1-REF', 'EEG A2-REF']

In [15]:
def feature_extraction(timeseries):
  # takes as an input one window in parallel with the windows from other channels
  #20 features
  features = []
  ##time domain
  # mean
  mean = np.mean(timeseries)
  # variance
  var = np.var(timeseries)
  # maximum
  max = np.max(timeseries)
  # minimum
  min = np.min(timeseries)
  # standard deviation
  std = np.std(timeseries)
  # line length   
  line_length = 0
  for i in range(0, len(timeseries)):
    if i == 0:
      line_length += abs(timeseries[i])
    else:
      line_length += abs(timeseries[i] - timeseries[i-1])  
  # zero-crossing
  zero_crossing = (np.diff(np.sign(timeseries)) != 0).sum()
  # kurtosis
  kurtosis = stats.kurtosis(timeseries)
  # skewness
  skew = stats.skew(timeseries)
  # median
  median = statistics.median(timeseries)
 
  ##frequency domain
  psd = matplotlib.mlab.psd(timeseries)
  power = psd[0].tolist()
  freqs = psd[1].tolist()

  low_freq = 0
  theta = 0
  alpha = 0
  beta = 0
  gamma = 0
  for i in range(0, len(psd[0])):
    # low frequency power
    if freqs[i] <= 0.04:
      low_freq += power[i]   
    #theta total power 4 - 8 Hz
    if 0.04 < freqs[i] <= 0.08:
      theta += power[i]
    # alpha total power
    if 0.08 < freqs[i] <= 0.12:
      alpha += power[i]
    # beta total power
    if 0.13 < freqs[i] <= 0.3:
      beta += power[i]
    # gamma total power
    if 0.3 < freqs[i] <= 1:
      gamma += power[i]
    
  # peak frequency
  #peak_freq = freqs[int(power.index(max(power)))]
  # spectral entropy
  ent = entropy(power)
  # magnitude of psd
  #mag_psd = matplotlib.mlab.magnitude_spectrum(timeseries)[0]# first array is magnitude, the second - frequencies
  # angle of spectrum
  #ang_psd = matplotlib.mlab.angle_spectrum(timeseries)[0]
  # phase of spectrum
  #ph_psd = matplotlib.mlab.phase_spectrum(timeseries)[0]

  features.append(mean)
  features.append(var)
  features.append(max)
  features.append(min)
  features.append(std)
  features.append(line_length)
  features.append(zero_crossing)
  features.append(kurtosis)
  features.append(skew)
  features.append(median)
  features.append(low_freq)
  features.append(theta)
  features.append(alpha)
  features.append(beta)
  features.append(ent)

  return features